In [ ]:
# comment out to run on drive + colab

! pip install transformers_interpret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 856.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# from google.colab import drive
#
# drive.mount('/content/drive/')
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers_interpret import MultiLabelClassificationExplainer
import torch
from transformers import pipeline
from datasets import load_from_disk

import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'transformers_interpret'

In [ ]:
# comment out the filepath to run on drive + colab
filepath = "../models/scenario_level/wernicke_control"
# filepath = "/content/drive/Shareddrives/AphasiaProject/models/scenario_level/wernicke_control"

model = AutoModelForSequenceClassification.from_pretrained(filepath+"/model")
tokenizer = AutoTokenizer.from_pretrained(filepath+"/tokenizer", padding=True,
                                          truncation=True, return_tensors="pt",
                                          add_special_tokens=True, max_length=512)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == torch.device("cuda"):
    model.to(device)
pipe = pipeline("text-classification", model=filepath+"/model", tokenizer=filepath+"/tokenizer", device=device, truncation=True, padding=True)
cls_explainer = MultiLabelClassificationExplainer(model, tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model.config

DistilBertConfig {
  "_name_or_path": "../models/wernicke_control/model",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "CONTROL",
    "1": "WERNICKE"
  },
  "initializer_range": 0.02,
  "label2id": {
    "CONTROL": 0,
    "WERNICKE": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.39.0.dev0",
  "vocab_size": 30524
}

In [ ]:
model.config.id2label

{0: 'CONTROL', 1: 'WERNICKE'}

In [ ]:
def interpret(data):
    for text, label in zip(list(data["preprocessed_text"]), list(data["label"])):
        if device == torch.device("cuda"):
            inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt", add_special_tokens=True, max_length=512).input_ids.to(device)
            # tranformers interpret gives error for input of 512 tokens
            if len(inputs[0]) != 512:
                with torch.no_grad():
                    logits = model(inputs).logits

                predicted_class_id = logits.argmax().item()
                print("True: ", label, "Pred: ", model.config.id2label[predicted_class_id])
                print("Text: ", text)
                cls_explainer(text)
                cls_explainer.visualize()
                print("----------------------------------------------------------------------")
        else:
            inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt", add_special_tokens=True, max_length=512)
            # tranformers interpret gives error for input of 512 tokens
            if len(inputs[0]) != 512:
                with torch.no_grad():
                    logits = model(**inputs).logits

                predicted_class_id = logits.argmax().item()
                print("True: ", label, "Pred: ", model.config.id2label[predicted_class_id])
                print("Text: ", text)
                cls_explainer(text)
                cls_explainer.visualize()
                print("----------------------------------------------------------------------")

In [ ]:
scenarios = ["Speech", "Important_Event", "Cinderella", "Stroke", "Cat"]
# make sure not to interpret on trained scenarios
texts = load_from_disk(filepath + "/dataset")["test"].remove_columns(["Unnamed: 0", "input_ids", "attention_mask"]).to_pandas()
texts["label"] = [model.config.id2label[x] for x in texts["label"]]
n = 3

# Speech scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [ ]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[0])].sample(n=n)])

In [ ]:
interpret(data)

True:  CONTROL Pred:  CONTROL
Text:  yes hindi.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.68),CONTROL,1.71,[CLS] yes hindi . [SEP]
n/a,(0.34),WERNICKE,-1.71,[CLS] yes hindi . [SEP]


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  okay i speak good english. may i speak indian with her? okay.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.73),CONTROL,1.97,[CLS] okay i speak good english . may i speak indian with her ? okay . [SEP]
n/a,(0.29),WERNICKE,-2.34,[CLS] okay i speak good english . may i speak indian with her ? okay . [SEP]


----------------------------------------------------------------------
True:  CONTROL Pred:  WERNICKE
Text:  i think [FP] i think it's fair. yes. tamil. tamil is the tamil is mostly i speak with my husband because he speaks my same the language i speak. to my children, [FP] half and half you know. a few words here and there but not much.


...

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.65),CONTROL,0.88,[CLS] my speech is my speech is today is not not very good . [SEP]
n/a,(0.34),WERNICKE,-1.80,[CLS] my speech is my speech is today is not not very good . [SEP]


----------------------------------------------------------------------
True:  WERNICKE Pred:  WERNICKE
Text:  very good. yes. it's always different. [FP] the talking they don't work. yeah.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.23),CONTROL,-3.22,[CLS] very good . yes . it ' s always different . [FP] the talking they don ' t work . yeah . [SEP]
n/a,(0.78),WERNICKE,3.21,[CLS] very good . yes . it ' s always different . [FP] the talking they don ' t work . yeah . [SEP]


----------------------------------------------------------------------
True:  WERNICKE Pred:  CONTROL
Text:  how how how good my speech is? [FP] pretty good. yeah. not not very good. it's just becoming just becoming active. and then we had restart.


----------------------------------------------------------------------


# Important event scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [ ]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[1])].sample(n=n)])

In [ ]:
interpret(data)

True:  CONTROL Pred:  CONTROL
Text:  i applied for a visa to the us. and my visa was rejected twice. and the second time they put on a stamp on my passport that i'm not eligible visa. but somehow i i presented my case through my high commissioner in england to the us ambassador. and they agreed to accept my application. and [FP] they immediately offered critical continual. and that was a milestone in my career.


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  it was such an immense change culturally. and we also we lived in a very [FP] rural area in new mexico up in the mountains. we lived in towes which is the biggest town we lived in which is not a very big town at all. so when we came here it was absolutely shocking. i mean just the fact that they had sidewalks alone. they just amazed us. and the street lamps and everything. i think one thing that's different about the rural areas in the west and the urban areas in the east is the east has so much more money. it's so much richer. of course there's the there's a lot [FP] [FP] sparser population. so a lot of times they're not gonna pave these huge long roads all over the place. but here it was just amazing to us, all the asphalt and concrete and steel and the immense size of the city. it was just indescribable the impact that it had on us. i think it took us years to really kind of o

----------------------------------------------------------------------
True:  WERNICKE Pred:  WERNICKE
Text:  it could be anything. i like. i like driving the car. [FP] i like going on driving. i i like anything. i bought i i i do anything. i [FP] will do anything. [FP] it's weird to say. i will do any. mhm. i got married. i got [FP] [FP] [FP] i heard a lot of things. it it's a quench. which one you want? oh ours was it's just like because our ours was just a separate ceremony. her. it's me and her. it's us. mhm.


----------------------------------------------------------------------
True:  WERNICKE Pred:  WERNICKE
Text:  oh. oh. oh [FP]. oh. well loyola university [FP] chicago [FP]. well i went to the university [FP] [FP] sixty two i think it look. downtown chicago avenue [FP] business school [FP] four years. i [FP] [FP] bachelor degree accounting. [FP] i well city [FP] well part time [FP] field [FP] two years then i i full full time two years. [FP] [FP] i i think so nine [FP] [FP] [FP] [FP] [FP] seven. no no. [FP] [FP] [FP] [FP] [FP] what year? [FP] not nineties but [FP] [FP] [FP] sixty yeah sixty six i graduated [FP] loyola. then i went to canada three years. then i came back. [FP] then university of chicago two years. [FP] masters degree. then i worked well [FP] ninety [FP] one i finish master of account [FP] masters degree. then i went to one year [FP] ninety [FP] [FP] seventy one seventy two i worked [FP] namep namep accounting firm. [FP] then i c p a i got the c p a. then i [FP] [FP] may 

----------------------------------------------------------------------


# Cinderella scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [ ]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[2])].sample(n=n)])

In [ ]:
interpret(data)

True:  CONTROL Pred:  CONTROL
Text:  was very jealous of cinderella. and she was [FP] taken to live with them. and they made a slave out of her. and they put her in every kind of a bad circumstance that they could. and she went ahead and did the work [FP] very jolly. and then [FP] she got to wanting to do something. so [FP] she wanted to go to this dance. [FP] and she was wishing for a prince to come and get her. and [FP] the [FP] fairy godmother waved a wand. and got a coach out of a pumpkin with a [FP] handsome prince [FP] in it. so he had a pair of glass slippers for her to wear. and if she wasn't back home by midnight well the coach would turn back to a pumpkin. and the prince would leave too. so when she was going late getting home. she ran off and lost one of the slippers. so the old man he found it. and he went around trying to find who it was that who had the slippers on. so [FP] he finally tried the [FP] wicked witch. and her feet was way too big. and her two daughters ugly da

----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  [FP] [UP] it may have been that she goes to this family under false pretenses because [FP] she winds up really as a servant to the mother and these two girls. both of whom are spoiled and rather mean to cinderella. [FP] [UP] the king announces that there's going to be a ball. and for i guess it's a ball the prince is expected to choose a woman to be his wife. [FP] the two girls in the family of course get themselves all dressed up with very high expectations that one of them might be chosen. [UP] as they go off to the ball the fairy godmother comes and dresses cinderella up. tells her she can go to the ball too. as a matter of fact gets a [FP] i guess a pumpkin and [FP] mice as i recall. which are turned into a horse and carriage. a very fancy horse and carriage. and cinderella goes off to the ball. and of course cinderella turns out to be the belle of the ball. the prince falls 

----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  her parents die when she's a little girl. her evil step mom comes in and takes her away to live with her. makes her the house slave. and [FP] she dreams of a better life. and eventually one day the trying to say king's courtesan comes to invite people to a party. gives the stepsisters an invitation. cinderella wants to go. mom says or stepmom says. if you get all the chores done you can go. so she does chores. is all excited. makes a dress with her little woodland friends and mice. her stepmom said she can't go. she cries. fairy godmother comes in. gives her the glass slippers and the crazy dress naturally. and [FP] she goes to the ball. tells her if she's not back by [FP] or if she doesn't get kissed by midnight she'll turn back into the house bitch. [FP], anyways, yeah. so she goes to the ball. meets prince charming. and dances and dances and dances. something something happens

----------------------------------------------------------------------
True:  WERNICKE Pred:  WERNICKE
Text:  i'll call her cin. cin. [FP] she's she's gonna be live with. i i don't know. this other family she's being she's gonna be with [FP] [FP] [FP] another family. but this new family are are very unhappy about [FP] her. not very good. and she hadta do and see or whatever. she she hadta do [FP] working and abc cooking, all kinds of work. [FP] every single day, work work work work work work work. she's doing everything. and [FP] then there's a ball. eventually there's gonna be a [FP] a b. let's see. a ball. but a ball a party ball. and [FP] a ball. and everyone was getting everyone was getting [FP] really really pretty to go. but she was not [FP] she was not allowed to go. [FP] she was not allowed to go. so everyone was getting ready. and then there was some. oh yeah there were also some animals that helped. jklm mice mice were gonna help her too. the godmother was [FP] was gonna help

----------------------------------------------------------------------
True:  WERNICKE Pred:  WERNICKE
Text:  and then [FP] [FP] let's see. and then [FP] yeah. okay [FP] and then page. [FP] [FP] and then [FP] laughing and talking mean man girl. and [FP] and then [FP] and then horse. and a nice girl and horse. and then [FP] laughing and talking [FP] a [FP] three girls mean man. [FP] and then [FP] and then [FP] cinderella and. oh boy! and then [FP] [FP] and then [FP] dancing. cinderella and another man and dancing. and then [FP] at twelve o'clock at midnight and. uhoh. and. [FP] [FP] oh boy. oh ah yeah. and then [FP] and then next day and then [FP] mean man tree mean man. and then sit down and [FP] slip [FP] slip [FP] perfect. and. ah. [FP] [FP] two girl and man married. and wow, lord.


----------------------------------------------------------------------


# Stroke scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [ ]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    df = texts[(texts["label"] == x) & (texts["scenario"] == scenarios[3])]
    # no controls in stroke scenario
    if len(df) >= n:
        data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[3])].sample(n=n)])

In [ ]:
interpret(data)

True:  WERNICKE Pred:  WERNICKE
Text:  and i was driving from down on the beach. and i drove home. and i'd got in the my in my house then i went in my bed. and i knew there was something wrong with me. and i don't know why but i must have put the phone and called my either one of my [FP] girls. and they came immediately with their father of of the one [FP] tom who is the father of one. get those people out of there. i mean i was awake of that. i said. please. and finally they said. well if you if we let us take you to the hospital right away we'll put it we'll call them away. the [FP] you know what it is, the bang bang bang and the whole thing. i can't make it better. so immediately we got to the cemetary the hospital. and when i said. i i i wanna go home and stay by myself and leave. so finally i did but i i was you know uncertain the way they treated me. and [FP] and then i had [FP] [FP] quite a few doctors that up time to find out. and [FP] that's what happened was it [FP] would be 

----------------------------------------------------------------------


# Cat scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [ ]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[4])].sample(n=n)])

In [ ]:
interpret(data)

True:  CONTROL Pred:  CONTROL
Text:  the kitty is up in the tree. the little girl it's obviously her kitty. she's got her hands out waiting for kitty. the man went up the tree apparently to get the kitty. the ladder has now fallen down. there's a dog down there barking. so the man can't jump down but the fire truck with the real leader has finally come to the rescue.


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  alright. little [FP] little suzanne here. kitty is stuck in the tree. and her dad is says. no problem honey. i i'll get the cat down out of the tree. and the dog's barking. so the cat's not gonna get down out of there. there's a dog at the bottom of the tree. he's barking. and so the cat's not gonna come down on his own. so dad climbs up the tree. and [FP] lo and behold the ladder falls down. now we have both the kitty cat and dad stuck in the tree. and the little girl she's kinda worried about the cat there. and dad's like. well i'm stuck in the tree. so a neighbor has actually seen all this transpire and has called up the fire department to come and get dad and the cat down from the tree. the end.


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  and the father goes up the tree to to rescue the the cat. and he goes up a ladder. and and while he's up there somehow or other the somehow or other the ladder fell off fell away from the tree. now he's now he's stuck. and [FP] the dog is barking. he's all excited. and a little girl [FP] i guess that's his daughter is trying to coax a cat to come down. and somebody had called the [FP] firemen to come out and rescue the father. and [FP] so they're they're on their way to to put up a ladder and and get the get the father down.


----------------------------------------------------------------------
True:  WERNICKE Pred:  WERNICKE
Text:  mess i don't. my goodness. he doesn't want neither one. well he lets he has some on. and she said. i want him out but i don't get it. i don't get it there. she goes i hafta go. he can't go up that to get him. and he can't he can't hold to get it. so he he got. look at the. oh the. what is it? this one. and this whether he said. we have no and we that and we that. oh look at this at. i don't know what's he doing. what this? i don't know what that doing. i dunno.


----------------------------------------------------------------------
True:  WERNICKE Pred:  WERNICKE
Text:  and the firemen the firemen firemen. an he's good get toes. so he's in [FP] in. so [FP] i don't know. he had.


----------------------------------------------------------------------
True:  WERNICKE Pred:  WERNICKE
Text:  okay. okay. okay. but then girl. [FP] well crawling [FP] cat. yeah cat. [FP] [FP] then girl [FP] then well father maybe up in the tree getting the cat but not reaching the the cat. [FP] then [FP] [FP] well not [FP] bicycle [FP] on the ground. ladder. oh dog [FP] oh [FP] [FP] [FP] dog [FP] [FP] on the ground. then a fire [FP] trier no fire truck arrived. then that [FP] cat i think so that cat and firemen were truck arrive. then that ladder [FP] that cat [FP] now well now [FP] that cat [FP] then going [FP] ladder finally coming down. i think so. but [FP] [FP] fire truck [FP] go. [FP] [FP] ladder. yeah. yeah yeah. yeah yeah.


----------------------------------------------------------------------
